In [15]:
from rtf_utils.moat_utils import MoatTile, format_json_newline, clean_row
from rtf_utils.gcp_utils import BigQuery, CloudStorage
from rtf_utils.dfa_utils import CampaignManagerReport,clean_dcm_file
from essence.analytics.platform import securedcredentials as secure_creds
import json
import time

In [11]:
with open("rtf_utils/moat_config_pixel.json") as json_file:
        config = json.load(json_file)

moat_token = 'IhSsMXBfH1eQBYaeYEYDJxot3Gs0uq9iaIUm3Qzf'
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)

## Moat

In [7]:
## cm 
cm_disp = MoatTile(2506,"moat_google_display",[22830945],"disp",False)
cm_vid = MoatTile(2698,"moat_google_video",[22830945],"vid",False)

## twitter
tw_disp = MoatTile(6195541,"moat_tw_display",[21017248,20969946,20970333,20969992],"disp",True)
tw_vid = MoatTile(6195543,"moat_tw_video",[20970543],"vid",True)

## facebook 
fb_dis = MoatTile(6195503,"moat_fb_display",[23843331338570586,23843331340260586],"disp",True)
fb_vid = MoatTile(8268,"moat_fb_video",[23843331336980586],"vid",True)

## ig
ig_disp = MoatTile(6188035,"moat_ig_display",[23843460042250586,23843460040730586,23843460042240586],"disp",True)
ig_vid = MoatTile(13120,"moat_ig_vid",[23843460042260586,23843460042270586,23843460042240586],"vid",True)

## YT
yt_trv = MoatTile(13386,"Essence Google TrueView",[7020493427],"vid",False)
yt_trp = MoatTile(6179366,"YT Reserve", [2520197687,2549731844,2550298543,2547410762],"vid",True)

In [12]:
def get_tile_data(tiles,start_date,end_date,moat_token):   
    """
    date format YYYYMMDD
    """    
    for tile in tiles:
        start_time = time.time()
        print("Starting Tile: {}".format(tile.name))
        tile.get_data(start_date,end_date,moat_token)
        elapsed_time = round(time.time() - start_time,2)
        print("Elapsed Time {}\n{} Rows".format(elapsed_time,len(tile.data)))

def clean_and_save_tile(tiles,prefix=None):
    saved_files = []
    for tile in tiles:
        if prefix:
             filename = prefix + "_" + tile.name + ".json"
        else:
            filename = tile.name + ".json"
        
        if tile.data != []:        
            with open(filename, "w") as f:
                rows_cleaned = [clean_row(row) for row in tile.data]       
                rows = [json.dumps(row) for row in rows_cleaned]
                row_str = '\n'.join(rows)
                f.write(row_str)
            saved_files.append(filename)
        else:
            pass
    return saved_files

In [ ]:
start_date = '20190701'
end_date = '20190825'
tiles = [cm_disp,cm_vid,fb_dis,fb_vid,yt_trv,yt_trp]
get_tile_data(tiles,start_date,end_date,)

In [16]:
start_date = '20190701'
end_date = '20190825'
get_tile_data([ig_disp,ig_vid],start_date,end_date,moat_token)

Starting Tile: moat_ig_display
Elapsed Time 42.18
102 Rows
Starting Tile: moat_ig_vid
Elapsed Time 40.48
102 Rows


In [19]:
saved_files = clean_and_save_tile([ig_disp,ig_vid],prefix="q3")

In [20]:
cs = CloudStorage(credentialsFromVault)
for file in saved_files:
    print(file)
    cs.upload_blob('rtf_staging',file,file,'filename')

q3_moat_ig_display.json
q3_moat_ig_vid.json


In [28]:
## can load multiple uris into one table but tiles are of diff scehemas??
## Load All Disp 
## handle different columns having opid with coelesce(regex(opid) etc etc)

## load Video
resp = bq.load_from_gcs('rtf_pixel_brand_report',
                 [ 'gs://rtf_staging/q3_Essence Google TrueView.json',
 'gs://rtf_staging/q3_YT Reserve.json'],
                 moat_vid_schema,
                "moat_vid_test",
                ext='json'
                )
## load Disp
resp = bq.load_from_gcs('rtf_pixel_brand_report',
                 "gs://rtf_staging/q3_moat_google_display.json",
                 moat_disp_schema,
                "g_disp",
                ext='json'
                )

In [38]:
## Report ID's
verification_report = CampaignManagerReport(credentialsFromVault,5096586,631165400)

## DCM - Reports

In [39]:
verification_report = CampaignManagerReport(credentialsFromVault,5096586,631165400)
video_player_size = CampaignManagerReport(credentialsFromVault,5096586,632709822)

verification_report.set_date_range("2019-07-01","2019-08-25")
video_player_size.set_date_range("2019-07-01","2019-08-25")

verification_report.run_report()
video_player_size.run_report()

verification_report.get_report_status()


file_name = verification_report.download_file()


from time import sleep
video_player_size.get_report_status()
while video_player_size.status != "REPORT_AVAILABLE":
    sleep(5)
    video_player_size.get_report_status()
    print(video_player_size.status)
filename = video_player_size.download_file()
clean_dcm_file('Bonito_Video_Player_Size.csv')

In [88]:
def clean_dcm_file(filename):
    data = []
    write = False
    with open(filename,'r') as f:
        reader = csv.reader(f, delimiter=',')

        for row in reader:
            if write == True:
                data.append(row)        
            elif row == ['Report Fields']:
                write = True

        if data[-1][0] == 'Grand Total:':
            data.pop()

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [90]:
classification = clean_dcm_file(file_name)

In [87]:
cs.upload_blob("rtf_staging","dcm_verification.csv","",mode="filename")

In [97]:
resp = bq.load_from_gcs('rtf_pixel_brand_report',
                'gs://rtf_staging/Bonito_Classification.csv',
                 verification_schema,
               "dcm_verification",
                skip_rows = 1
                )

In [10]:
video_player_size.set_date_range("2019-07-01","2019-08-25")
video_player_size.run_report()

In [12]:
from time import sleep
video_player_size.get_report_status()
while video_player_size.status != "REPORT_AVAILABLE":
    sleep(5)
    video_player_size.get_report_status()
    print(video_player_size.status)
filename = video_player_size.download_file()
clean_dcm_file('Bonito_Video_Player_Size.csv')

PROCESSING
PROCESSING
PROCESSING
PROCESSING
REPORT_AVAILABLE


In [20]:
cs.upload_blob("rtf_staging","Bonito_Video_Player_Size.csv","Bonito_Video_Player_Size.csv",mode="filename")

In [24]:
resp = bq.load_from_gcs('rtf_brand_reporting',
                'gs://rtf_staging/Bonito_Video_Player_Size.csv',
                 player_size_schema,
               "DCM_playerSize",
                ext='csv',
                        skip_rows = 1
                )

In [25]:
resp = bq.load_from_gcs('rtf_brand_reporting',
                'gs://rtf_staging/Bonito_Classification.csv',
                 verification_schema,
               "DCM_contentVerification",
                skip_rows = 1
                )

In [33]:
## load Video
resp = bq.load_from_gcs('rtf_brand_reporting',
                 [ 'gs://rtf_staging/q3_Essence Google TrueView.json',
 'gs://rtf_staging/q3_YT Reserve.json'],
                 moat_vid_schema,
                "MOAT_videoTiles_RAW",
                ext='json'
                )
## load Disp
resp = bq.load_from_gcs('rtf_brand_reporting',
                 "gs://rtf_staging/q3_moat_google_display.json",
                 moat_disp_schema,
                "MOAT_displayTiles_RAW",
                ext='json'
                )

In [4]:
materialize_verification = """
                            SELECT
                              date,
                              CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                              SUM(verifiable_impressions) verifiable_impressions_no_issues
                            FROM
                              `essence-analytics-dwh.rtf_brand_reporting.DCM_contentVerification`
                            WHERE
                              classifier = "None Detected"
                            GROUP BY
                              1,
                              2

                            """

materialize_player_report = """
                            WITH
                              player_pivot AS (
                              SELECT
                                date,
                                CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                                CASE
                                  WHEN video_player_size = "HD" THEN verifiable_impressions
                              END
                                AS hd_player_impressions,
                                CASE
                                  WHEN video_player_size = "LARGE" THEN verifiable_impressions
                              END
                                AS large_player_impressions,
                                CASE
                                  WHEN video_player_size = "SMALL" THEN verifiable_impressions
                              END
                                AS small_player_impressions,
                                CASE
                                  WHEN video_player_size = "(not set)" THEN verifiable_impressions
                              END
                                AS player_not_set_impressions
                              FROM
                                `essence-analytics-dwh.rtf_brand_reporting.DCM_playerSize`)
                            SELECT
                              date,
                              opid,
                              SUM(hd_player_impressions) hd_player_impressions,
                              SUM(large_player_impressions) large_player_impressions,
                              SUM(small_player_impressions) small_player_impressions,
                              SUM(player_not_set_impressions) player_not_set_impressions
                            FROM
                              player_pivot
                            GROUP BY
                              1,
                              2
                            """


moat_vid = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level4_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS moat_vid_impressions_analyzed,
              SUM(susp_valid) AS moat_vid_susp_valid,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(reached_first_quart_sum) reached_first_quart_sum,
              SUM(reached_second_quart_sum) reached_second_quart_sum,
              SUM(reached_third_quart_sum) reached_third_quart_sum,
              SUM(reached_complete_sum) reached_complete_sum,
              SUM(player_visible_on_complete_sum) player_visible_on_complete_sum,
              SUM(player_audible_on_complete_sum) player_audible_on_complete_sum,
              SUM(player_vis_and_aud_on_complete_sum) player_vis_and_aud_on_complete_sum,
              SUM(susp_valid_and_inview_gm_meas_sum) susp_valid_and_inview_gm_meas_sum,
              sum(_5_sec_in_view_impressions) _5_sec_in_view_impressions,
              sum(susp_bot_geo_perc) susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_videoTiles_RAW`

            Group by 1,2
            """

helper_sql.moat_disp = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level3_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS impressions_analyzed,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(iva) AS iva,
              SUM(susp_bot_geo_perc) AS susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_displayTiles_RAW`
            GROUP BY
              1,
              2
            """



In [3]:
bq = BigQuery(credentialsFromVault)

In [9]:
resp = bq.store_query_results(materialize_verification,"rtf_brand_reporting","DCM_verificationReport_TRANSFORMED")

C:\Users\kyle.randolph\AppData\Local\Continuum\anaconda3\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [12]:
resp = bq.store_query_results(materialize_player_report,"rtf_brand_reporting","DCM_playerSize_TRANSFORMED")

In [14]:
resp = bq.store_query_results(moat_vid,"rtf_brand_reporting","MOAT_videoTiles_TRANSFORMED")

In [11]:
resp = bq.store_query_results(moat_disp,"rtf_brand_reporting","MOAT_displayTiles_TRANSFORMED")

AttributeError: module 'helper_sql' has no attribute 'moat_disp'

In [2]:
q

NameError: name 'q' is not defined